<h3 style='color:orange'> Importação das Bibliotecas </h3>

In [23]:
import connections
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 500)

In [24]:
connection = connections.get_db()

## Obtenção dos dados

In [4]:
df_raw = pd.read_csv('hostel_detail_data.csv',sep=';')

In [5]:
df_raw.columns

Index(['id_hostel', 'name', 'currency', 'type', 'maxNumberOfGuestsPerBooking',
       'isFeatured', 'latestCheckOut', 'rating', 'totalRatings', 'starRating',
       'facilities', 'description', 'latitude', 'longitude', 'address1',
       'address2', 'directions', 'depositPercentage', 'checkIn.startsAt',
       'checkIn.endsAt', 'city.id', 'city.name', 'city.country',
       'city.idCountry', 'region.id', 'region.name', 'payments.payNow',
       'cancellationPolicy.depositOnly.description',
       'cancellationPolicy.depositOnly.numberOfDays',
       'cancellationPolicy.nonRefundable.description',
       'cancellationPolicy.freeCancellation.description',
       'freeCancellation.isAvailable', 'freeCancellation.daysInAdvance',
       'groupInformation.groupMaxPax', 'taxInfo.included', 'taxInfo.taxes'],
      dtype='object')

### Remover as colunas desnecessárias

In [6]:
colunas_drop = ['rating',
                'directions',
                'groupInformation.groupMaxPax',
                'cancellationPolicy.nonRefundable.description',
                'cancellationPolicy.nonRefundable.description',
                'address2',
                'cancellationPolicy.freeCancellation.description',
                'cancellationPolicy.depositOnly.numberOfDays',
                'taxInfo.taxes',
                'maxNumberOfGuestsPerBooking',
                'isFeatured',
                'city.id',
                'city.idCountry',
                'region.id',
                'payments.payNow',
                'cancellationPolicy.depositOnly.description'
               ]

df = df_raw.drop(columns = colunas_drop)

### Renomear as colunas

In [7]:
df.rename(columns = {
        'latestCheckOut':   'last_checkout',
        'totalRatings':     'qtd_rating',
        'starRating':       'min_rating',
        'address1':         'address',
        'depositPercentage':'deposit_percentage',
        'checkIn.startsAt': 'checkin_start',
        'checkIn.endsAt':   'checkin_end',
        'city.name':        'city',
        'city.country':     'country',
        'region.name':      'region',
        'freeCancellation.isAvailable': 'free_cancelation_is_available',
        'freeCancellation.daysInAdvance': 'free_cancelation_days_in_advance',
        'taxInfo.included': 'tax_is_included'
    }, inplace = True
)

In [8]:
df['facilities'] = df['facilities'].apply(eval)

## Normalizações

<h3 style='color:orange'> Categoria de Facilidade </h3>

In [9]:
# Explodir o DataFrame para obtenção de todas as Categorias
df_exploded = df[['id_hostel','facilities']].explode('facilities').reset_index(drop = True)

# Usar o Json_Normalize para criar o DataFrame
df_category_raw = pd.json_normalize(df_exploded['facilities'])

# Adicionar o Id do Hostel
df_category_raw.insert(0, 'id_hostel', df_exploded['id_hostel'])

In [10]:
# Criar o DataFrame somente com as Categorias Existentes
df_category = df_category_raw[['numericId','name','id']].drop_duplicates()

# Remover registros Nulos
df_category = df_category.dropna()

# Ajustaro  tipo de dados para INT
df_category['numericId'] = df_category['numericId'].astype(int)

# Renomear as colunas
df_category.rename(columns = {
    'numericId':'id_category',
    'name':'category',
}, inplace = True)

# Remover colunas desnecessarias
df_category.drop(columns = ['id'], inplace = True)

#### Inserir no Banco de dados

In [12]:
df_category.to_sql('hostelworld_category_facilities', connection, index = False, if_exists = 'append')

5

<h3 style='color:orange'> Facilidades </h3>

In [13]:
# Explodir em mais uma camada para obter as facilidades das categorias
df_exploded = df_category_raw[['id_hostel','numericId','facilities']].explode('facilities').reset_index(drop = True)

# Criar o Novo DataFrame contendo as Facilidades
df_facilities_raw  = pd.json_normalize(df_exploded['facilities'])

In [14]:
# Remover os Registros Nulos
df_facilities_raw = df_facilities_raw.dropna()

# Inserir as colunas de Identificação do Hostel e da Categoria da Facilidade
df_facilities_raw.insert(0,'id_category', df_exploded['numericId'])
df_facilities_raw.insert(0,'id_hostel', df_exploded['id_hostel'])

In [15]:
# Criar o DataFrame das Facilidades que existem
df_facilities = df_facilities_raw[['numericId','name','id']].drop_duplicates()

# Remover os Nulos
df_facilities = df_facilities.dropna()

# Ajustar os tipos de dados
df_facilities['numericId'] = df_facilities['numericId'].astype(int)

# Renomear as colunas
df_facilities.rename(columns = {
    'numericId':'id_facility',
    'name':'facility'
}, inplace = True)

### Inserir a tabela de Facilidades no Banco

In [16]:
df_facilities

,id_facility,facility,id
0,80,Free City Maps,FREECITYMAPS
1,90,Free WiFi,FREEWIFI
2,87,Towels Included,TOWELSINCLUDED
3,37,Breakfast Not Included,BREAKFASTNOTINCLUDED
4,27,Elevator,ELEVATOR
...,...,...,...
3075,113,Jobs Board,JOBSBOARD
3586,144,Sauna,SAUNA
7552,143,Golf Course,GOLFCOURSE
11730,302,UKRAINEREFUGEES,UKRAINEREFUGEES


In [17]:
df_facilities.to_sql('hostelworld_facilities', connection, index = False, if_exists = 'append')

103

<h3 style='color:orange'> Tabela Fato </h3>

In [18]:
# Renomear as colunas
df_facilities_raw.rename(columns = {
    'numericId': 'id_facility'
}, inplace = True)

# Ajustar os tipos de dados
df_facilities_raw['id_category'] = df_facilities_raw['id_category'].astype(int)
df_facilities_raw['id_facility'] = df_facilities_raw['id_facility'].astype(int)

# Remover colunas não usadas
df_facilities_raw.drop(columns = ['name','id'], inplace = True)

### Inserir a tabela Fato no Banco

In [19]:
df_facilities_raw

,id_hostel,id_category,id_facility
0,177051,1,80
1,177051,1,90
2,177051,1,87
3,177051,2,37
4,177051,2,27
...,...,...,...
887910,300001,3,4
887911,300001,3,15
887912,300001,3,13
887913,300001,4,125


In [21]:
df_facilities_raw.to_sql('hostelworld_hostel_facilities', connection, index = False, if_exists = 'append', method = 'multi')

887375